# 🎙️ VoiceAid Health — Akan ASR: WER Evaluation

Evaluates `dennis-9/whisper-small_Akan_non_standardspeech` using **Word Error Rate (WER)**.

**Before running:**
- Enable GPU: **Runtime → Change runtime type → T4 GPU**
- Ensure these exist on your Google Drive:
  - `MyDrive/VoiceAid_Dataset/akan_dataset.xlsx`
  - `MyDrive/VoiceAid_Dataset/audio/` (folder with all .wav files)

In [ ]:
# Cell 1: Install Dependencies
!pip install -q transformers torch torchaudio jiwer openpyxl accelerate pandas
print('✅ Dependencies installed!')

In [ ]:
# Cell 2: Mount Google Drive & Set Paths
from google.colab import drive
import os

drive.mount('/content/drive')
print('✅ Google Drive mounted!')

# ── PATHS (update if your folder names differ) ──────────────────
AUDIO_DIR     = '/content/drive/MyDrive/VoiceAid_Dataset/audio/'
DATASET_EXCEL = '/content/drive/MyDrive/VoiceAid_Dataset/akan_dataset.xlsx'
# ────────────────────────────────────────────────────────────────

wav_files = [f for f in os.listdir(AUDIO_DIR) if f.endswith('.wav')]
print(f'🎵 Found {len(wav_files)} .wav files in {AUDIO_DIR}')

In [ ]:
# Cell 3: Load Dataset
import pandas as pd

df = pd.read_excel(DATASET_EXCEL)
print(f'✅ Dataset loaded: {len(df)} rows')
print(f'   Columns: {list(df.columns)}')
print(f'\n   Severity breakdown:')
print(df['Severity'].value_counts())
print(f'\n   Sample rows:')
df[['file_name', 'text', 'Severity', 'condition']].head(5)

In [ ]:
# Cell 4: Load Your Custom Akan Model
import torch
from transformers import pipeline

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'🖥️  Device: {device}')

MODEL_ID = 'dennis-9/whisper-small_Akan_non_standardspeech'
print(f'📥 Loading model: {MODEL_ID}...')

pipe = pipeline(
    'automatic-speech-recognition',
    model=MODEL_ID,
    device=device,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)
print('✅ Model loaded!')

In [ ]:
# Cell 5: Run WER Evaluation
from jiwer import wer, cer

# ── CONFIG ──────────────────────────────────────────────────────
SAMPLE_SIZE     = 100   # Start small. Set to len(df) for full eval (~30-60 mins)
SEVERITY_FILTER = None  # 'Mild', 'Moderate', 'Severe', or None for all
# ────────────────────────────────────────────────────────────────

if SEVERITY_FILTER:
    eval_df = df[df['Severity'] == SEVERITY_FILTER].sample(
        min(SAMPLE_SIZE, len(df[df['Severity'] == SEVERITY_FILTER])), random_state=42
    )
else:
    eval_df = df.sample(min(SAMPLE_SIZE, len(df)), random_state=42)

print(f'🧪 Evaluating {len(eval_df)} samples (Severity: {SEVERITY_FILTER or "All"})...')

references, hypotheses, errors = [], [], []

for idx, row in eval_df.iterrows():
    audio_path = os.path.join(AUDIO_DIR, str(row['file_name']))
    if not os.path.exists(audio_path):
        errors.append(f'Missing: {row["file_name"]}')
        continue
    try:
        result = pipe(audio_path, generate_kwargs={
            'max_new_tokens': 256,
            'temperature': 0.0,
            'num_beams': 5,
        })
        references.append(str(row['text']))
        hypotheses.append(result['text'].strip())
    except Exception as e:
        errors.append(f'Error on {row["file_name"]}: {e}')

overall_wer = wer(references, hypotheses)
overall_cer = cer(references, hypotheses)

print(f'\n{"="*50}')
print(f'📊 EVALUATION RESULTS ({len(references)} samples)')
print(f'{"="*50}')
print(f'  Word Error Rate  (WER): {overall_wer:.2%}  ← Lower is better')
print(f'  Char Error Rate  (CER): {overall_cer:.2%}  ← Lower is better')
print(f'  Accuracy (approx):      {(1-overall_wer):.2%}')
print(f'  Missing/Errored:        {len(errors)}')
print(f'{"="*50}')

print('\n📝 Sample Predictions:')
for ref, hyp in list(zip(references, hypotheses))[:5]:
    print(f'  REF: {ref}')
    print(f'  HYP: {hyp}\n')

In [ ]:
# Cell 6: WER Breakdown by Severity Level
from jiwer import wer

print('📊 WER by Severity Level')
print('='*40)

for severity in sorted(df['Severity'].dropna().unique()):
    sev_df = df[df['Severity'] == severity].sample(
        min(50, len(df[df['Severity'] == severity])), random_state=42
    )
    sev_refs, sev_hyps = [], []
    for _, row in sev_df.iterrows():
        audio_path = os.path.join(AUDIO_DIR, str(row['file_name']))
        if not os.path.exists(audio_path): continue
        try:
            result = pipe(audio_path, generate_kwargs={'max_new_tokens': 256, 'temperature': 0.0})
            sev_refs.append(str(row['text']))
            sev_hyps.append(result['text'].strip())
        except: continue

    if sev_refs:
        sev_wer = wer(sev_refs, sev_hyps)
        bar = '█' * int((1 - sev_wer) * 20)
        print(f'  {severity:12s}: WER = {sev_wer:.2%}  [{bar:<20}] ({len(sev_refs)} samples)')

In [ ]:
# Cell 7: Save Results to Google Drive
import json

results = {
    'model': MODEL_ID,
    'sample_size': len(references),
    'overall_wer': round(overall_wer, 4),
    'overall_cer': round(overall_cer, 4),
    'accuracy_approx': round(1 - overall_wer, 4),
    'severity_filter': SEVERITY_FILTER or 'All',
    'missing_or_errored': len(errors),
}

RESULTS_PATH = '/content/drive/MyDrive/VoiceAid_Dataset/wer_results.json'
with open(RESULTS_PATH, 'w') as f:
    json.dump(results, f, indent=2)

print(f'✅ Results saved to: {RESULTS_PATH}')
print(json.dumps(results, indent=2))